<a href="https://colab.research.google.com/github/Nishthamaybeme/House-Price-Prediction/blob/main/house_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd

# Assuming your train dataset is loaded into a DataFrame called 'train_data'
# Example:
train_data = pd.read_csv('/content/drive/MyDrive/housing price prediction/train.csv')

# Separate categorical columns from numeric columns
categorical_columns = train_data.select_dtypes(include=['object']).columns

# Apply one-hot encoding to categorical columns
train_data_encoded = pd.get_dummies(train_data, columns=categorical_columns, drop_first=True)

# Check the transformed data
print(train_data_encoded.head())


   Id  MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
0   1          60         65.0     8450            7            5       2003   
1   2          20         80.0     9600            6            8       1976   
2   3          60         68.0    11250            7            5       2001   
3   4          70         60.0     9550            7            5       1915   
4   5          60         84.0    14260            8            5       2000   

   YearRemodAdd  MasVnrArea  BsmtFinSF1  ...  SaleType_ConLI  SaleType_ConLw  \
0          2003       196.0         706  ...           False           False   
1          1976         0.0         978  ...           False           False   
2          2002       162.0         486  ...           False           False   
3          1970         0.0         216  ...           False           False   
4          2000       350.0         655  ...           False           False   

   SaleType_New  SaleType_Oth  SaleTyp

In [ ]:
# Assuming your test dataset is loaded into a DataFrame called 'test_data'
# Example:
test_data = pd.read_csv('/content/drive/MyDrive/housing price prediction/test.csv')

# Identify categorical columns in the training data
categorical_columns = train_data.select_dtypes(include=['object']).columns

# Apply one-hot encoding to the test data, using the same categorical columns as in train data
test_data_encoded = pd.get_dummies(test_data, columns=categorical_columns, drop_first=True)

# Ensure the test data has the same columns as the train data (if any categories are missing in the test data)
# Align columns by adding missing columns with NaN values
test_data_encoded = test_data_encoded.reindex(columns=train_data_encoded.columns, fill_value=0)

# Check the transformed test data
print(test_data_encoded.head())


     Id  MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  \
0  1461          20         80.0    11622            5            6   
1  1462          20         81.0    14267            6            6   
2  1463          60         74.0    13830            5            5   
3  1464          60         78.0     9978            6            6   
4  1465         120         43.0     5005            8            5   

   YearBuilt  YearRemodAdd  MasVnrArea  BsmtFinSF1  ...  SaleType_ConLI  \
0       1961          1961         0.0       468.0  ...           False   
1       1958          1958       108.0       923.0  ...           False   
2       1997          1998         0.0       791.0  ...           False   
3       1998          1998        20.0       602.0  ...           False   
4       1992          1992         0.0       263.0  ...           False   

   SaleType_ConLw  SaleType_New  SaleType_Oth  SaleType_WD  \
0           False         False         False         True  

In [ ]:
print("Columns in train dataset:")
print(train_data.columns)

print("\nColumns in test dataset:")
print(test_data.columns)

Columns in train dataset:
Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
      

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# Load the one-hot encoded datasets

# Save Id from the test data, it will be needed later in the output
if 'Id' in test_data.columns:
    test_id = test_data['Id']
else:
    test_id = None
    print("No 'Id' column found in the test data.")

# Separate the target variable 'SalePrice' from the features
X_train = train_data.drop(columns=['SalePrice'], errors='ignore')
y_train = train_data['SalePrice']

# Handle missing values in numeric columns for both train and test datasets
numeric_columns = X_train.select_dtypes(include=['float64', 'int64']).columns

# Fill missing values with the median of each column
X_train[numeric_columns] = X_train[numeric_columns].fillna(X_train[numeric_columns].median())
X_test = test_data.copy()  # Make a copy of the test data to predict on

# Fill missing values in the test dataset
X_test[numeric_columns] = X_test[numeric_columns].fillna(X_test[numeric_columns].median())

# Handle missing values in categorical columns for both train and test datasets
categorical_columns = X_train.select_dtypes(include=['object']).columns

# Fill missing values in categorical columns with the mode (most frequent value)
X_train[categorical_columns] = X_train[categorical_columns].fillna(X_train[categorical_columns].mode().iloc[0])
X_test[categorical_columns] = X_test[categorical_columns].fillna(X_test[categorical_columns].mode().iloc[0])

# Now, ensure that the categorical columns are one-hot encoded in both datasets
X_train = pd.get_dummies(X_train, drop_first=True)  # One-hot encode and drop the first column to avoid multicollinearity
X_test = pd.get_dummies(X_test, drop_first=True)

# Align the columns of X_train and X_test to have the same columns
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

# Train the model (Random Forest Regressor in this case)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict the SalePrice for the test dataset
y_pred = model.predict(X_test)

# Ensure the output includes both Id and predicted SalePrice
if test_id is not None:
    output = pd.DataFrame({'Id': test_id, 'SalePrice': y_pred})
else:
    output = pd.DataFrame({'SalePrice': y_pred})

# Specify the file path for saving predictions
output_path = '/content/drive/MyDrive/housing_price_predictions_output.csv'
output.to_csv(output_path, index=False)

print(f"Predictions saved to '{output_path}'")


Predictions saved to '/content/drive/MyDrive/housing_price_predictions_output.csv'
